<a href="https://colab.research.google.com/github/VincentBarnes/Git-Test/blob/master/langg_working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install required packages
!pip install langgraph langchain-core


In [2]:
# Import libraries
from langgraph.graph import Graph, END
from langchain_core.tools import tool
from typing import TypedDict, Annotated, List
import operator
import json

In [16]:
# 1. Define REAL Tools (not mocks)
@tool
def multiply(a: float, b: float) -> float:
    """Multiply two numbers together. Input must be two numbers."""
    return a * b

@tool
def add(a: float, b: float) -> float:
    """Add two numbers together. Input must be two numbers."""
    return a + b

@tool
def get_word_length(word: str) -> int:
    """Get the length of a word. Input must be a single word."""
    return len(word)

tools = {t.name: t for t in [multiply, add, get_word_length]}


In [17]:
# 2. Create REAL Agent Logic
class Agent:
    def __init__(self, tools):
        self.tools = tools

    def decide_action(self, prompt: str):
        """Analyzes the prompt and decides whether to use tools or respond directly"""
        prompt = prompt.lower()

        # Check if prompt matches any tool's purpose
        for tool_name, tool in self.tools.items():
            if any(keyword in prompt for keyword in tool.description.lower().split()):
                # Try to extract parameters from prompt
                try:
                    if tool_name == "get_word_length":
                        word = prompt.split()[-1].strip(",.?!")
                        return {
                            "action": "tool",
                            "tool": tool_name,
                            "input": {"word": word}
                        }
                    elif tool_name in ["multiply", "add"]:
                        nums = [float(s) for s in prompt.split() if s.replace('.','',1).isdigit()]
                        if len(nums) >= 2:
                            return {
                                "action": "tool",
                                "tool": tool_name,
                                "input": {"a": nums[0], "b": nums[1]}
                            }
                except:
                    continue

        # Fallback to direct response
        if "moon" in prompt:
            return {"action": "response", "content": "The Moon is Earth's natural satellite."}
        return {"action": "response", "content": "I need more specific information to help with that."}

agent = Agent(tools)


In [18]:
# 3. Define Agent State
class AgentState(TypedDict):
    user_input: str
    agent_output: str
    next_step: str
    steps: Annotated[List[str], operator.add]
    tool_data: dict


In [19]:
# 4. Define Graph Nodes
def call_agent(state: AgentState):
    decision = agent.decide_action(state["user_input"])

    if decision["action"] == "tool":
        return {
            "agent_output": "",
            "next_step": "use_tool",
            "tool_data": decision,
            "steps": state["steps"] + [f"Decided to use {decision['tool']} tool"]
        }
    else:
        return {
            "agent_output": decision["content"],
            "next_step": "end",
            "tool_data": {},
            "steps": state["steps"] + ["Responding directly"]
        }

def call_tool(state: AgentState):
    tool_name = state["tool_data"]["tool"]
    tool_input = state["tool_data"]["input"]
    result = tools[tool_name].invoke(tool_input)
    return {
        "agent_output": str(result),
        "next_step": "end",
        "tool_data": {},
        "steps": state["steps"] + [f"Tool result: {result}"]
    }


In [20]:
# 5. Build and Run the Graph
workflow = Graph()
workflow.add_node("agent", call_agent)
workflow.add_node("tool", call_tool)
workflow.set_entry_point("agent")

workflow.add_conditional_edges(
    "agent",
    lambda state: state["next_step"],
    {"use_tool": "tool", "end": END}
)
workflow.add_edge("tool", END)

app = workflow.compile()


In [21]:
# 6. Test with REAL Inputs
test_inputs = [
    "Multiply 7 by 8",
    "What's the length of hello?",
    "Add 15 and 25",
    "Tell me about the moon",
    "What's the meaning of life?"
]

for input_text in test_inputs:
    print(f"\nQuestion: {input_text}")

    state = {
        "user_input": input_text,
        "agent_output": "",
        "next_step": "",
        "steps": [],
        "tool_data": {}
    }

    output = app.invoke(state)

    print("Thought process:")
    for step in output["steps"]:
        print(f"- {step}")
    print(f"\nFinal Answer: {output['agent_output']}")
    print("-" * 50)


Question: Multiply 7 by 8
Thought process:
- Decided to use multiply tool
- Tool result: 56.0

Final Answer: 56.0
--------------------------------------------------

Question: What's the length of hello?
Thought process:
- Decided to use get_word_length tool
- Tool result: 5

Final Answer: 5
--------------------------------------------------

Question: Add 15 and 25
Thought process:
- Decided to use add tool
- Tool result: 40.0

Final Answer: 40.0
--------------------------------------------------

Question: Tell me about the moon
Thought process:
- Decided to use get_word_length tool
- Tool result: 4

Final Answer: 4
--------------------------------------------------

Question: What's the meaning of life?
Thought process:
- Decided to use get_word_length tool
- Tool result: 4

Final Answer: 4
--------------------------------------------------
